In [7]:
import os
os.chdir("C:\\Users\\joona\\ylikool\\baka\\pcb-detector\\code\\training\\yolo_dataset\\")
os.listdir()

['classes.txt',
 'data.yaml',
 'images',
 'labels',
 'notes.json',
 'train.txt',
 'val.txt',
 'yolov11_keypoint_train']

In [ ]:
import os
import glob
import numpy as np
import tensorflow as tf
import cv2
from tensorflow.keras.utils import Sequence
import keras.utils
from keras.applications.vgg16 import VGG16
from keras.layers import Input, Flatten, Dense, Dropout, Conv2D, MaxPooling2D, BatchNormalization, DepthwiseConv2D
from keras.models import Model, Sequential
from keras.optimizers import Adam, SGD, Adadelta

# Dataset directory
dataset_dir = "C:\\Users\\joona\\ylikool\\baka\\pcb-detector\\code\\training\\yolo_dataset\\"

# Define data generator class
class KeypointDataGenerator(Sequence):
    def __init__(self, image_files, label_files, batch_size=1, img_size=(320, 320)):
        self.image_files = image_files
        self.label_files = label_files
        self.batch_size = batch_size
        self.img_size = img_size
        self.indexes = np.arange(len(self.image_files))

    def __len__(self):
        return int(np.floor(len(self.image_files) / self.batch_size))

    def __getitem__(self, index):
        batch_indexes = self.indexes[index * self.batch_size: (index + 1) * self.batch_size]
        batch_images = [self.load_image(self.image_files[i]) for i in batch_indexes]
        batch_keypoints = [self.load_keypoints(self.label_files[i]) for i in batch_indexes]

        batch_images = np.array(batch_images)
        batch_keypoints = np.array(batch_keypoints)

        return batch_images, batch_keypoints

    def load_image(self, image_path):
        img = cv2.imread(image_path)
        img = cv2.resize(img, self.img_size)
        img = img / 255.0  # Normalize
        return img

    def load_keypoints(self, label_path):
        keypoints = []
        with open(label_path, "r") as f:
            for line in f.readlines():
                parts = line.strip().split()
                if len(parts) < 4:
                    continue
                _, x, y, _ = map(float, parts)
                keypoints.append([x, y])

        max_kp_length = 32  # Adjust based on dataset
        keypoints = np.array(keypoints).flatten()
        keypoints = np.pad(keypoints, (0, max_kp_length - len(keypoints)), mode="constant")
        return keypoints

# Load dataset paths
train_images = sorted(glob.glob(os.path.join(dataset_dir, "images/*.jpg")))
train_labels = sorted(glob.glob(os.path.join(dataset_dir, "labels/*.txt")))

# Create generator with batch_size=1
train_gen = KeypointDataGenerator(train_images, train_labels, batch_size=1)

# Define a simple model (Modify as needed)
model = tf.keras.models.Sequential()
#https://github.com/VL97/Pose-estimation-using-Deep-CNN-encoder/blob/master/train.py
model.add(Input(shape=(1, 320, 320, 3)))
model.add(Conv2D(96, (11, 11), activation='relu', padding='same'))
model.add(Dropout(0.4))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(256, (5, 5), activation='relu', padding='same'))
model.add(Dropout(0.4))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(384, (3, 3), activation='relu', padding='same'))
model.add(Dropout(0.2))
model.add(Conv2D(384, (3, 3), activation='relu', padding='same'))
model.add(Dropout(0.5))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Dropout(0.2))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())

# Dense layers
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.6))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.6))

# Output layer: predicts keypoints (x,y pairs)
#max_kp_length = y_train.shape[1]  # Max padded length from dataset
model.add(Dense(32, activation='sigmoid'))  # Normalize keypoints to [0,1]

model.compile(optimizer='adam', loss='mse')

# Train model using generator
model.fit(train_gen, epochs=30, verbose=1)


C:\Users\joona\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
136/136 ━━━━━━━━━━━━━━━━━━━━ 887s 6s/step - loss: 0.2695
Epoch 2/30
136/136 ━━━━━━━━━━━━━━━━━━━━ 878s 6s/step - loss: 0.2546
Epoch 3/30
136/136 ━━━━━━━━━━━━━━━━━━━━ 882s 6s/step - loss: 0.2467
Epoch 4/30
136/136 ━━━━━━━━━━━━━━━━━━━━ 881s 6s/step - loss: 0.2590
Epoch 5/30
136/136 ━━━━━━━━━━━━━━━━━━━━ 882s 6s/step - loss: 0.2674
Epoch 6/30
136/136 ━━━━━━━━━━━━━━━━━━━━ 877s 6s/step - loss: 0.2603
Epoch 7/30
136/136 ━━━━━━━━━━━━━━━━━━━━ 888s 7s/step - loss: 0.2624
Epoch 8/30
136/136 ━━━━━━━━━━━━━━━━━━━━ 884s 6s/step - loss: 0.2520
Epoch 9/30
136/136 ━━━━━━━━━━━━━━━━━━━━ 881s 6s/step - loss: 0.2616
Epoch 10/30
136/136 ━━━━━━━━━━━━━━━━━━━━ 879s 6s/step - loss: 0.2638
Epoch 11/30
136/136 ━━━━━━━━━━━━━━━━━━━━ 884s 6s/step - loss: 0.2508
Epoch 12/30
136/136 ━━━━━━━━━━━━━━━━━━━━ 883s 6s/step - loss: 0.2613
Epoch 13/30
136/136 ━━━━━━━━━━━━━━━━━━━━ 882s 6s/step - loss: 0.2670
Epoch 14/30
136/136 ━━━━━━━━━━━━━━━━━━━━ 880s 6s/step - loss: 0.2685
Epoch 15/30
136/136 ━━━━━━━━━━━━━━━━━━━━ 88

In [11]:
model.save("aaa.keras")

In [12]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_5 (Conv2D)               │ (None, 320, 320, 96)   │        34,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 320, 320, 96)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 160, 160, 96)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 160, 160, 256)  │       614,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 160, 160, 256)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 80, 80, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 80, 80, 384)    │       885,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 80, 80, 384)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 80, 80, 384)    │     1,327,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 80, 80, 384)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 80, 80, 256)    │       884,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 80, 80, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 40, 40, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 409600)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1024)           │   419,431,424 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1024)           │     1,049,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 32)             │        32,800 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,272,783,074 (4.74 GB)

 Trainable params: 424,261,024 (1.58 GB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 848,522,050 (3.16 GB)

In [17]:
os.listdir()

['2b7a221a-image61.jpg',
 'classes.txt',
 'data.yaml',
 'images',
 'labels',
 'notes.json',
 'train.txt',
 'val.txt',
 'yolov11_keypoint_train']

In [26]:

def load_image(image_path):
        img_size=(320, 320)
        img = cv2.imread(image_path)
        print(img)
        img = cv2.resize(img, img_size)
        img = img / 255.0  # Normalize
        return img

In [27]:

tulemus= model.predict(np.expand_dims(load_image('2b7a221a-image61.jpg'), axis=0))
print(tulemus)


[[[ 43  46  44]
  [ 42  47  45]
  [ 44  53  50]
  ...
  [116 110  97]
  [121 117  99]
  [112 111  91]]

 [[ 40  45  43]
  [ 39  46  43]
  [ 38  47  44]
  ...
  [114 108  95]
  [115 111  93]
  [115 114  94]]

 [[ 40  51  48]
  [ 38  49  46]
  [ 39  50  47]
  ...
  [115 109  96]
  [123 119 101]
  [117 113  95]]

 ...

 [[  0   7   7]
  [  0   6   5]
  [  0   8   7]
  ...
  [ 11  28  37]
  [  6  22  34]
  [  7  23  36]]

 [[  0   9  12]
  [  0   8   8]
  [  0   8   5]
  ...
  [  9  26  35]
  [ 10  26  42]
  [  8  24  41]]

 [[  0   8  12]
  [  0   8  11]
  [  0   9   6]
  ...
  [  6  23  32]
  [  7  23  40]
  [  7  22  41]]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 347ms/step
[[0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]]
